<a href="https://colab.research.google.com/github/MadhuvanthiThanikachalam/Machine-learning/blob/main/Information%20retrieval_Decision%20tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import string
import scipy
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('always')  # This makes it so that each warning is only shown once. 

from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [ ]:
def encode(source):
	le = LabelEncoder()
	center = np.array(source['a4'])
	classes = le.fit_transform(source[['class']])
	class_inverted = le.inverse_transform((classes))
	class_map = {}

	for i in range(len(classes)):
	    if class_inverted[i] not in class_map:
	        class_map[class_inverted[i]] = [classes[i], 1]
	    else:
	        class_map[class_inverted[i]][1] += 1
	
	lowest = float('inf') #Set it to a theoretical upper bound
	lowest_class = ""
	
	for key in class_map: #Find the lowest
		if class_map.get(key)[1] < lowest:
			lowest = class_map.get(key)[1]
			lowest_class = class_map.get(key)[0]
	
			print(class_map)
			print(class_map.get(key)[1])
	
	word_dict = {}
	final_list = []

	for i in range(len(center)):
		if center[i] not in word_dict:
			word_dict[center[i]] = classes[i]

	selected_attributes = source[['a1', 'a2', 'a3', 'a4', 'a5', 'a6']].to_numpy() #Use the leave one out method here by removing attributes.

	for row in selected_attributes: 
		local_list = []
		for i in row:
			if i in word_dict:
				local_list.append(word_dict.get(i))
			else:
				local_list.append(lowest_class)
		final_list.append(local_list)


	x = scipy.sparse.csr_matrix(final_list)
	y = scipy.sparse.csr_matrix(classes)

	return x,y #return two compressed sparse row matrices with unique value IDs for each class.



In [ ]:
def trainer(classifier, x_train, x_test, y_train, y_test):


    classifier.fit(x_train.todense(), y_train)
    y_pred = classifier.predict(x_test.todense())


    f1 = f1_score(y_test, y_pred, average="macro")
    precision = precision_score(y_test, y_pred, average="macro", labels=np.unique(y_pred)) #Unique labels that fixes a y_pred error
    recall = recall_score(y_test, y_pred, average="macro", labels=np.unique(y_pred))
    accuracy = accuracy_score(y_test, y_pred)
    return f1, precision, recall, accuracy


In [ ]:
def trainDTC(x,y):
	f1List = list()
	precisionList = list()
	recallList = list()
	accuracyList= list()
	
	# n_splits = 5, we are using 5-fold cross validation
	kf = KFold(n_splits = 5)

	for train_index, test_index in kf.split(x):
		x_train, x_test = x[train_index], x[test_index]
		y_train = []
		y_test = []
		for i in train_index:
			y_train.append(y[0, i])
		for i in test_index:
			y_test.append(y[0, i])
		f1, precision, recall, accuracy= trainer(DecisionTreeClassifier(random_state=0), x_train, x_test, y_train, y_test)
		f1List.append(f1)
		precisionList.append(precision)
		recallList.append(recall)
		accuracyList.append(accuracy)


	averageAccuracy = np.mean(accuracyList)
	averagePrecision = np.mean(precisionList)
	averageF1 = np.mean(f1List)
	averageRecall = np.mean(recallList)
	print("Decision Tree Classifier Average Accuracy for: {}".format(averageAccuracy))
	print("Decision Tree Classifier Average Precision for: {}".format(averagePrecision))
	print("Decision Tree Classifier Average F1 for: {}".format(averageF1))
	print("Decision Tree Classifier Average Recall for: {}".format(averageRecall))

In [ ]:
def build():
	data = pd.read_csv('pos-eng-5000.data.csv')
	print(data)
	x, y = encode(data)
	trainDTC(x,y)


build()

             a1         a2         a3  ...       a6      a7   class
0             _          _          _  ...  remains  closed      DT
1             _          _        The  ...   closed  PERIOD      NN
2             _        The  cafeteria  ...   PERIOD       _     VBZ
3           The  cafeteria    remains  ...        _       _      JJ
4     cafeteria    remains     closed  ...        _       _  PERIOD
...         ...        ...        ...  ...      ...     ...     ...
4995       side         to       side  ...   frenzy  PERIOD      IN
4996         to       side         in  ...   PERIOD       _      DT
4997       side         in          a  ...        _       _      NN
4998         in          a     frenzy  ...        _       _  PERIOD
4999          _          _          _  ...     seem      to     PRP

[5000 rows x 8 columns]
{'DT': [8, 447], 'NN': [16, 648], 'VBZ': [37, 93], 'JJ': [12, 328], 'PERIOD': [21, 234], 'NNS': [19, 269], 'VBD': [33, 222], 'IN': [11, 487], 'EX': [9, 12], 'M

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Decision Tree Classifier Average Accuracy for: 0.9513999999999999
Decision Tree Classifier Average Precision for: 0.8867489898977441
Decision Tree Classifier Average F1 for: 0.8528961722781855
Decision Tree Classifier Average Recall for: 0.910607069696631


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
